In [1]:
from langchain_groq import ChatGroq

In [ ]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='api_key', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-43372")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Senior Principal AI/ML Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Principal AI/ML EngineerBeaverton, OregonBecome a Part of the NIKE, Inc. TeamN

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Principal AI/ML Engineer',
 'experience': '10+ years of experience across multiple roles in developing productionized code in software engineering, machine learning or a related field (experience delivering analytics and ML products preferred). 4+ years in a leadership or management role using Agile frameworks.',
 'skills': ['Python',
  'algorithms and data structures',
  'AWS',
  'databases',
  'data processing technologies',
  'ETL, ML, or analytics technologies such as Scikit-learn, Dask, Tensorflow, Kubeflow, Spark, EMR, or similar platforms and frameworks',
  'open-source technologies',
  'MLOps, code management workflows, infrastructure as code, test-driven development, and the lifecycle of model development from experimentation to production and measurement'],
 'description': 'As a Senior Principal AI/ML Engineer, you will lead the development of robust machine learning and generative AI solutions that have a direct impact on the business. You are a seasoned Eng

In [9]:
type(json_res)

dict

In [11]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [14]:
job = json_res
job['skills']

['Python',
 'algorithms and data structures',
 'AWS',
 'databases',
 'data processing technologies',
 'ETL, ML, or analytics technologies such as Scikit-learn, Dask, Tensorflow, Kubeflow, Spark, EMR, or similar platforms and frameworks',
 'open-source technologies',
 'MLOps, code management workflows, infrastructure as code, test-driven development, and the lifecycle of model development from experimentation to production and measurement']

In [16]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [19]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Nitin, a developer Xelso. Xelso is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Xelso 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Nitin. Developer at Xelso
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert AI/ML Solutions for Your Business Needs

Dear Hiring Manager,

I came across the job description for a Senior Principal AI/ML Engineer and was impressed by the scope of the role. As a seasoned developer at Xelso, an AI & Software Consulting company, I believe we can provide the expertise and support required to fulfill your business needs.

With over a decade of experience in developing productionized code in software engineering, machine learning, and related fields, our team is well-equipped to lead the development of robust machine learning and generative AI solutions. Our expertise in Python, algorithms and data structures, AWS, databases, and data processing technologies aligns perfectly with the requirements of the role.

We have a proven track record of delivering analytics and ML products using ETL, ML, or analytics technologies such as Scikit-learn, Dask, Tensorflow, Kubeflow, Spark, EMR, or similar platforms and frameworks. Our experience with open-source tech